# News search using Hopsworks and Langchain

In this tutorial, you will learn how to create a news search bot which can answer users' question about news using Opensearch in Hopsworks with Langchain. Concretely, you will create a RAG (Retrieval-Augmented Generation) application which searches news matching users' questions, and answers the question using a LLM with the retrieved news as the context.
The steps include:
1. [Ingest news data to Hopsworks](https://github.com/logicalclocks/hopsworks-tutorials/blob/master/api_examples/hsfs/knn_search/news-search-knn.ipynb)
2. Setup a `vectorstores` in Langchain using Opensearch in Hopsworks
3. Create a LLM using model from huggingface
4. Create a RAG application using `RetrievalQA` chain in Langchain

## Prerequisite

You need to run this [notebook](https://github.com/logicalclocks/hopsworks-tutorials/blob/master/api_examples/hsfs/knn_search/news-search-knn.ipynb) to ingest news data to Hopsworks.

## Setup a vector store in Langchain using Opensearch in Hopsworks

First, you need to get the opensearch configuration, and the index name in the vector store from Hopsworks.

In [ ]:
import hopsworks
from hsfs.core.opensearch_api import OpenSearchApi
proj = hopsworks.login()
fs = proj.get_feature_store()

opensearch_config = OpenSearchApi(project_id=proj.id, project_name=proj.name).get_default_py_config()
opensearch_config["opensearch_url"] = f'{opensearch_config["hosts"][0]["host"]}:{opensearch_config["hosts"][0]["port"]}'
opensearch_config.pop("hosts")

# `news_fg.embedding_index.index_name` return the index name
news_fg = fs.get_feature_group(
    name="news_fg",
    version=1,
)

Then, you can setup the vector store in Langchain using the configuration, and the embedding model used for generating the embedding in the feature group.

In [ ]:
from langchain_community.vectorstores import OpenSearchVectorSearch
from langchain.embeddings import SentenceTransformerEmbeddings

embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

docsearch = OpenSearchVectorSearch(
    index_name=news_fg.embedding_index.index_name,
    embedding_function=embeddings,
    **opensearch_config
)


## Create a LLM using model from huggingface

You need to load a llm model from huggingface. You can pick any model on huggingface. To accelerate the inference, you can use load the model to gpu if available.

In [ ]:
from langchain.llms import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch

import torch
# Check for GPU availability and set device
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available!")
else:
    device = torch.device("cpu")
    print("GPU is not available, using CPU.")

    
# Load the Llama2 chat model (replace with your preferred model name)
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0" 
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, torch_dtype=torch.bfloat16, device=device)
llm = HuggingFacePipeline(pipeline=pipe)

# Create a RAG application using `RetrievalQA` chain in Langchain

Lastly, you need to create a prompt for the llm, and create a `RetrievalQA` chain in Langchain. You need to provide `vector_field`, and `text_field` which are feature names in the `news_fg` feature group. You can also modify the number of results returned from the vector store by adjusting `k`.

In [ ]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA

# Prompt
template = """
<|system|>
Use the following pieces of context to answer the question from user. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
context:
{context} </s>
<|user|>
{question}</s>
<|assistant|>
"""
QA_CHAIN_PROMPT = PromptTemplate(
    input_variables=["context", "question"],
    template=template,
)

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=docsearch.as_retriever(
        search_kwargs={
            "vector_field": news_fg.embedding_body.name, 
            "text_field": news_fg.article.name, 
            "k": 1}
    ),
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT},
)



In [ ]:
question = "any news about France?"
result = qa_chain({"query": question})
print(result["result"])